# Opcode Sequence을 사용한 machineLearning 기반 malware detection Model
- dataset: https://github.com/iosifache/DikeDataset
- accuracy: 0.992169507139567

# 필요 package 설치

In [1]:
!pip install pefile
!pip install capstone

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# utils.py 파일, sample 파일이 있는 폴더로 위치 변경

In [2]:
%cd /content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test

/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/휴지통/test


# 필요 package Import

In [3]:
from utils import *

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from xgboost import XGBClassifier

## opcode 리스트 로드
- file 폴더에 있는 opcodesList.txt 파일을 filePath로 설정하시면 됩니다.

In [4]:
filePath = '/content/drive/MyDrive/Capstone Design Project + 한이음 과정/code/files/opcode database/opcodesList.txt'
with open(filePath, 'rb') as lf:
    opcodes = pickle.load(lf)
    print(opcodes)

['aaa', 'aad', 'aam', 'aas', 'adc', 'add', 'and', 'call', 'cbw', 'clc', 'cld', 'cli', 'cmc', 'cmp', 'cmpsb', 'cmpsw', 'cwd', 'daa', 'das', 'dec', 'div', 'esc', 'hlt', 'idiv', 'imul', 'in', 'inc', 'int', 'into', 'iret', 'jcc', 'ja', 'jae', 'jb', 'jbe', 'jc', 'je', 'jg', 'jge', 'jl', 'jle', 'jnle', 'jno', 'jnp', 'jns', 'jnz', 'jo', 'jp', 'jpe', 'jpo', 'js', 'jz', 'jcxz', 'jmp', 'lahf', 'lds', 'lea', 'les', 'lock', 'lodsb', 'lodsw', 'loop', 'loope', 'loopne', 'loopnz', 'loopz', 'loopx', 'mov', 'movsb', 'movsw', 'mul', 'neg', 'nop', 'not', 'or', 'out', 'pop', 'popf', 'push', 'pushf', 'rcl', 'rcr', 'repxx', 'rep', 'repe', 'repne', 'repnz', 'repz', 'ret', 'retn', 'retf', 'rol', 'ror', 'sahf', 'sal', 'sar', 'sbb', 'scasb', 'scasw', 'shl', 'shr', 'stc', 'std', 'sti', 'stosb', 'stosw', 'sub', 'test', 'wait', 'xchg', 'xlat', 'xor']


# 학습된 XGBClassifier 모델 로드

# 모델 로드

In [5]:
# 파일명
filename = 'XGB_Word2vecClassifier.model'

# 모델 불러오기
clf = pickle.load(open(filename, 'rb'))

# word2vec 모델 로드
modelPath = 'w2v'
word2vecModel = KeyedVectors.load_word2vec_format(modelPath)

# word2vec 임베딩 값을 위한 함수
- word2vec 임베딩 값의 평균을 임베딩 값으로 사용

In [6]:
def get_sentence_mean_vector(morphs):
    vector = []
    for i in morphs:
        try:
            vector.append(word2vecModel.wv[i])
        except KeyError as e:
            pass
    try:
        return np.array(np.mean(vector, axis=0), dtype = np.float64)
    except IndexError as e:
        pass

## MalwareDetectionFunction을 통한 Malware 탐지
- 양성 샘플 파일이 양성으로 결과가 나오는지 확인
- return 1: 악성코드, return 0: 양성코드

In [7]:
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
# 머신러닝 모델(XGBClassifier)을 바탕으로 해당 파일이 악성코드인지를 알려주는 함수
# return 1: 악성코드, return 0: 양성코드
def MalwareDetectionFunction(model, fileName):
    try:
        #parse exe file
        exe = pefile.PE(fileName)
        #call the function we created earlier
        sampleOpcodeList = ExtractPefileOpcodes(opcodes, exe)
    except:
        print('Error File')
    
    inputData = get_sentence_mean_vector(sampleOpcodeList)
    inputData = [inputData]

    return clf.predict(inputData)[0]

- 결과가 0으로 양성파일을 탐지한 것읋 확인할 수 있다.

In [8]:
fileName = '3f3fe9ecad7f30fc80cdfb678d7ca27a30d0575a73818746e98be9170d3be348.exe'

MalwareDetectionFunction(clf, fileName)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


0